In [91]:
import numpy as np
import os
import pickle

sz_float = 4
epsilon = 10e-8

# Getting rec_ids

In [88]:
data_path = '../data/test_data/'
labels_path = '../data/test_labels/test_tree.pckl'
records = dict()

In [94]:
for record in os.listdir(data_path):
    if not record.endswith('.bin'): continue
    f = open(os.path.join(data_path, record))
    records[int(record[:-4])] = (os.path.join(data_path, record),os.fstat(f.fileno()).st_size/sz_float)
    f.close()

In [96]:
sf = 0
rec_ids = (2556, 2303, 2382, 2416, 2191, 2298, 2106, 2628, 1819, 1759)
window = 16384
m = 128
offset = 44100
normalize=True


with open(labels_path, 'rb') as f:
    labels = pickle.load(f)

In [97]:
def access(rec_id,s,shift=0,jitter=0):
    """
    Args:
        rec_id (int): MusicNet id of the requested recording
        s (int): Position of the requested data point
        shift (int, optional): Integral pitch-shift data transformation
        jitter (float, optional): Continuous pitch-jitter data transformation
    Returns:
        tuple: (audio, target) where target is a binary vector indicating notes on at the center of the audio.
    """

    scale = 2.**((shift+jitter)/12.)

    fid,_ = records[rec_id]
    with open(fid, 'rb') as f:
        f.seek(s*sz_float, os.SEEK_SET)
        x = np.fromfile(f, dtype=np.float32, count=int(scale*window))

    if normalize: x /= np.linalg.norm(x) + epsilon

    xp = np.arange(window,dtype=np.float32)
    x = np.interp(scale*xp,np.arange(len(x),dtype=np.float32),x).astype(np.float32)

    y = np.zeros(m,dtype=np.float32)
    for label in labels[rec_id][s+scale*window/2]:
        y[label.data[1]+shift] = 1

    return x,y

In [156]:
count = 7500
for i in range(len(rec_ids)): # loop for num of songs
    X = np.zeros([count,1, window, 1])
    Y = np.zeros([count, m])
    
    stride = (records[rec_ids[i]][1] - offset - int(sf*window))/count
    stride = int(stride)
    
    for j in range(count):
        X[j, 0, :,0], Y[j] = access(rec_ids[i], offset+j*stride)
        
    break
    

In [154]:
def visualize_labels(Y, offset, seconds=30, fs = 44100, stride = 512):            
    wps = fs/float(stride)               # calculate windows/second
    Yvec = np.zeros((int(seconds*wps),128))   # 128 distinct note labels
#     colors = {1 : .3a3}
##### To do #####
# make 1 instrument 1 color
#################

    for window in range(Yvec.shape[0]):
        labels = Y[offset+window*stride]
        for label in labels:
            Yvec[window,label.data[1]] = 1

    fig = plt.figure(figsize=(20,5))
    plt.imshow(Yvec.T,aspect='auto',cmap='ocean_r')
    plt.gca().invert_yaxis()
    fig.axes[0].set_xlabel('window')
    fig.axes[0].set_ylabel('note (MIDI code)')
#     plt.close()
    
    return Yvec

SyntaxError: 'break' outside loop (cell_name, line 26)